Advent of Code, Day 3

As usual, we'll start with reading in our data file. In this case, the only initial manipulation we'll need is to strip the trailing whitespace.

In [ ]:
with open('data.csv') as f:
    lines = [l.strip() for l in f.readlines()]

Next, we'll begin preparing our data structure. We don't necessarily need to know which rucksack belongs to which Elf, but at this point, we can presume that's a likely requirement, so let's plan ahead. We'll employ basic string splicing and set theory to quickly narrow in on what item is the common item between the rucksack compartments.

In [ ]:
elv_id = 0
rucksacks = {}

for l in lines:
    rucksacks[elv_id] = frozenset(l[:len(l)//2]).intersection(l[len(l)//2:])
    elv_id +=1

Next, we'll need to create our prioritization data. We can enumerate the integer values for each of the lower and upper case letter sets.

In [ ]:
from string import ascii_lowercase, ascii_uppercase
lower_case_priorities = {v: i for i,v in enumerate(ascii_lowercase, 1)}
upper_case_priorities = {v: i for i,v in enumerate(ascii_uppercase, len(lower_case_priorities) + 1)}

priorities = lower_case_priorities | upper_case_priorities

Finally, we can go through and add up the values of each our our common rucksack items. Of note here is the unpacking of the `frozenset` in order to use the value as the lookup key for the priorities dict. Let's create a general function we can call to compute these values in case we need it again.

In [ ]:
def compute_ruck_value(sacks, value=0):
    for r in sacks.values():
        item, = r
        value += priorities[item]

    return value

compute_ruck_value(rucksacks)

Part Two

In part two, we'll need to make groups of three elves based on the order of the lines in the input file. itertools provides a recipe for just this case. We'll use this `batched` recipe to create our groups.

In [ ]:
import itertools

def batched(iterable, n):
    "Batch data into lists of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := list(itertools.islice(it, n))):
        yield batch

groups = batched(lines, 3)

With our groups ready to go, we next need to massage our rucksack contents into a form that will allow us to find the common item. We use list comprehension on our batch in order to make each item in the group a set so that we may easily find the intersection (i.e. the common item).

In [ ]:
elv_groups = {}
elv_group_id = 0

for batch in groups:
    if elv_group_id in elv_groups:
        elv_groups[elv_group_id].append(batch)
    else:
        elv_groups[elv_group_id] = [set(b) for b in batch]
    elv_group_id += 1

The `badges` are the common item we will be looking for, using a combination of list expansion and dictionary comprehension to quickly get through all of our elv groups.

In [ ]:
badges = {k:set.intersection(*v) for (k,v) in elv_groups.items()}

Finally, as expected, we can use our general compute ruck value function to calculate the sum of the contents based on their priority scores.

In [ ]:
compute_ruck_value(badges)